In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
files = glob("/home/shrey/CSE8803EPI/Project/Google_Search_Trends_Data/*.csv")
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
del dfs